In [6]:
import os, shutil
os.makedirs("submission/data", exist_ok=True)
shutil.move("/content/cli_qa.json", "submission/data/cli_qa.json")


'submission/data/cli_qa.json'

In [12]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [1]:
!pip uninstall -y bitsandbytes
!pip install -q transformers datasets accelerate peft



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.5 MB/s eta 0:00:00


In [7]:
import json
with open("submission/data/cli_qa.json") as f:
    data = json.load(f)

print(f"Loaded {len(data)} Q&A pairs.")


Loaded 155 Q&A pairs.


In [3]:
# STEP 1: Install Required Libraries
!pip install -q transformers datasets accelerate peft

# STEP 2: Load JSON Dataset
import os, json
from datasets import Dataset

with open("submission/data/cli_qa.json") as f:
    data = json.load(f)

hf_dataset = Dataset.from_list(data)


In [4]:
# STEP 3: Tokenizer + Label Preparation
from transformers import AutoTokenizer

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)

def tokenize(example):
    prompt = f"Q: {example['question']}\nA: {example['answer']}"
    tokens = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = hf_dataset.map(tokenize, remove_columns=["question", "answer"])


Map:   0%|          | 0/155 [00:00<?, ? examples/s]

In [5]:
# STEP 4: Load Model (NO device_map to avoid meta tensor errors)
import torch
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained(model_id).to("cuda" if torch.cuda.is_available() else "cpu")


In [6]:
# STEP 5: Apply LoRA Adapter
from peft import get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05
)

model = get_peft_model(base_model, peft_config)


In [7]:
# STEP 6: Define TrainingArguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="submission/training/adapter_model",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    remove_unused_columns=False,
    report_to="none"  # Disable wandb and other loggers
)


In [8]:
# STEP 7: Train the Model
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

# Save the final LoRA adapter weights
model.save_pretrained("submission/training/adapter_model")


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,6.259700
20,2.662600
30,0.951500
40,0.395400
50,0.196900
60,0.158300
70,0.146400


In [9]:
!zip -r adapter_model.zip submission/training/adapter_model


  adding: submission/training/adapter_model/ (stored 0%)
  adding: submission/training/adapter_model/checkpoint-78/ (stored 0%)
  adding: submission/training/adapter_model/checkpoint-78/rng_state.pth (deflated 25%)
  adding: submission/training/adapter_model/checkpoint-78/training_args.bin (deflated 52%)
  adding: submission/training/adapter_model/checkpoint-78/adapter_model.safetensors (deflated 8%)
  adding: submission/training/adapter_model/checkpoint-78/scaler.pt (deflated 60%)
  adding: submission/training/adapter_model/checkpoint-78/README.md (deflated 66%)
  adding: submission/training/adapter_model/checkpoint-78/trainer_state.json (deflated 67%)
  adding: submission/training/adapter_model/checkpoint-78/scheduler.pt (deflated 56%)
  adding: submission/training/adapter_model/checkpoint-78/optimizer.pt (deflated 7%)
  adding: submission/training/adapter_model/checkpoint-78/adapter_config.json (deflated 54%)
  adding: submission/training/adapter_model/adapter_model.safetensors (def

In [10]:
from google.colab import files
files.download("adapter_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>